Notes:
- line_profiler is used for relative speeds of lines, but slows down the overall execution, that's why iteration speed is measured separately


- TODO: improve var names, e.g. "numba" for numba, etc.
- TODO: add markdown notes and include them in the report. i.e. generate a markdown report that can paste into blog. (need to be able to run in command line because faster than jupyter)


Idea:
- maybe faster to pass in questions and users seperately, rather than scores matrix, which is large... idk

In [ ]:
import io
import itertools
import math
import time
from math import sqrt
from typing import Iterable

import line_profiler
import numba
import numpy as np
import pandas as pd
from pandas import IndexSlice as islice


K = 5
DEFAULT_MAX_ITER = 1000
BEST_OF = 3
IS_ITER_SCALED = False  # scale up the number of iterations if fast

In [ ]:
data = pd.read_json("data/large.json")

In [ ]:
def profile_func(outer_func: callable, func_to_profile: callable) -> str:
    lp = line_profiler.LineProfiler()
    outer_func_with_profiler = lp(outer_func)
    lp.add_function(func_to_profile)
    outer_func_with_profiler()
    stream = io.StringIO()
    lp.print_stats(stream=stream)
    report = stream.getvalue()
    # Format report string
    s = ""
    # Get first table of report
    add = False
    for line in report.split("\n"):
        if f"Function: {func_to_profile.__name__}" in line:
            add = True
        if add:
            s += line + "\n"
        if add and "Total time" in line:
            break
    # Get only the % Time column onwards
    start_idx = s.find("% Time")
    for line in s.split("\n"):
        if "Line #" in line:
            start_idx = line.index("% Time")
            break
    s = "\n".join(x[start_idx:] for x in s.split("\n"))
    return s


def fmt_seconds(seconds: float) -> str | None:
    units = [
        (1e-9, "ns"),
        (1e-6, "μs"),
        (1e-3, "ms"),
        (1, "sec"),
    ]
    for threshold, unit in reversed(units):
        if seconds > threshold:
            time_in_unit = seconds / threshold
            if time_in_unit > 100:
                time_in_unit = 10 * round(time_in_unit / 10)
            elif time_in_unit > 10:
                time_in_unit = round(time_in_unit)
            else:
                time_in_unit = round(time_in_unit * 10) / 10
            return f"{time_in_unit} {unit}"

# Original code - baseline

In [ ]:
def k_corrset(data, max_iter, K):
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    q_to_score = data.set_index(["question", "user"])
    grand_totals = data.groupby("user").score.sum().rename("grand_total")
    start = time.time()
    corrs = compute_corrs(qs_iter, q_to_score, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(
    qs_iter: Iterable, q_to_score: pd.DataFrame, grand_totals: pd.DataFrame
):
    result = []
    for qs in qs_iter:
        qs_data = q_to_score.loc[islice[qs, :], :].swaplevel()
        answered_all = qs_data.groupby(level=[0]).size() == K
        answered_all = answered_all[answered_all].index
        qs_total = (
            qs_data.loc[islice[answered_all, :]]
            .groupby(level=[0])
            .sum()
            .rename(columns={"score": "qs"})
        )
        r = qs_total.join(grand_totals).corr().qs.grand_total
        result.append({"qs": qs, "r": r})
    return result


print(f"Computing corrs_baseline, avg_iter_time_secs_baseline...")
avg_iter_time_secs_baseline = math.inf
for _ in range(BEST_OF):
    corrs_baseline, avg_iter_time_secs = k_corrset(data, max_iter=DEFAULT_MAX_ITER, K=K)
    avg_iter_time_secs_baseline = min(avg_iter_time_secs_baseline, avg_iter_time_secs)

In [ ]:
benchmark_count = 0


def run_benchmark(
    k_corrset_func,
    compute_corrs_func,
    description,
    num_iter=None,
    run_line_profiler=True,
):
    global benchmark_count
    benchmark_count += 1
    print(f"Benchmark #{benchmark_count}: {description}")

    # Test values for correctness, and possibly warmup jit
    corrs, _ = k_corrset_func(data, K=K, max_iter=DEFAULT_MAX_ITER)
    assert np.allclose(corrs_baseline.r, corrs.r, equal_nan=True)

    if num_iter is None and IS_ITER_SCALED:
        # scale number of iters based on speedup, for more reliable timing
        _, avg_iter_time_secs = k_corrset_func(data, K=K, max_iter=DEFAULT_MAX_ITER)
        multiplier = avg_iter_time_secs_baseline / avg_iter_time_secs
        num_iter = int(DEFAULT_MAX_ITER * multiplier)
    num_iter = num_iter or DEFAULT_MAX_ITER
    print(f"Using {num_iter} iterations...")

    # Measure iteration time
    best_avg_iter_time_secs = math.inf
    for _ in range(BEST_OF):
        _, avg_iter_time_secs = k_corrset_func(data, K=K, max_iter=num_iter)
        best_avg_iter_time_secs = min(best_avg_iter_time_secs, avg_iter_time_secs)
    print(f"\nAvg time per iteration:  {fmt_seconds(best_avg_iter_time_secs)}")
    print(
        f"Speedup over baseline:   {avg_iter_time_secs_baseline/best_avg_iter_time_secs:0.1f}x"
    )
    print()

    if run_line_profiler:
        s = profile_func(
            lambda: k_corrset_func(data, K=K, max_iter=DEFAULT_MAX_ITER),
            compute_corrs_func,
        )
        print(s.strip())
        print()


run_benchmark(k_corrset, compute_corrs, "Baseline")

In [ ]:
def k_corrset(data, K, max_iter=1000):
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = {q: set() for q in data.question.unique()}
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q].add(u)
    q_to_score = data.set_index(["question", "user"])
    grand_totals = data.groupby("user").score.sum().rename("grand_total")
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, q_to_score, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_iter, users_who_answered_q, q_to_score, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = [users_who_answered_q[q] for q in qs]
        answered_all = set.intersection(*user_sets_for_qs)
        qs_data = q_to_score.loc[islice[qs, :], :].swaplevel()
        qs_total = (
            qs_data.loc[islice[list(answered_all), :]]
            .groupby(level=[0])
            .sum()
            .rename(columns={"score": "qs"})
        )
        r = qs_total.join(grand_totals).corr().qs.grand_total
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "Dict of sets to identify users who answered qs",
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = {q: set() for q in data.question.unique()}
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q].add(u)
    score_dict = {
        (q, u): s
        for q, u, s in data[["question", "user", "score"]].itertuples(index=False)
    }
    grand_totals = data.groupby("user").score.sum().rename("grand_total")
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = [users_who_answered_q[q] for q in qs]
        answered_all = set.intersection(*user_sets_for_qs)
        qs_total = {u: sum(score_dict[q, u] for q in qs) for u in answered_all}
        qs_total = pd.DataFrame.from_dict(qs_total, orient="index", columns=["qs"])
        qs_total.index.name = "user"
        r = qs_total.join(grand_totals).corr().qs.grand_total
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "Dict to look up scores",
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = {q: set() for q in data.question.unique()}
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q].add(u)
    score_dict = {
        (q, u): s
        for q, u, s in data[["question", "user", "score"]].itertuples(index=False)
    }
    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = [users_who_answered_q[q] for q in qs]
        answered_all = set.intersection(*user_sets_for_qs)
        qs_total = [sum(score_dict[q, u] for q in qs) for u in answered_all]
        user_grand_total = [grand_totals[u] for u in answered_all]
        r = np.corrcoef(qs_total, user_grand_total)[0, 1]
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(k_corrset, compute_corrs, "Dict to look up grand totals")

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = {q: set() for q in data.question.unique()}
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q].add(u)
    score_dict = {
        (q, u): s
        for q, u, s in data[["question", "user", "score"]].itertuples(index=False)
    }
    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = [users_who_answered_q[q] for q in qs]
        answered_all = set.intersection(*user_sets_for_qs)
        qs_total = [sum(score_dict[q, u] for q in qs) for u in answered_all]
        user_grand_total = [grand_totals[u] for u in answered_all]
        r = np.corrcoef(qs_total, user_grand_total)[0, 1]
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(k_corrset, compute_corrs, "Replacing long string UUIDs with ints")

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = np.zeros(
        (len(data.question.unique()), len(data.user.unique())), dtype=np.bool_
    )
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q, u] = True
    score_dict = {
        (q, u): s
        for q, u, s in data[["question", "user", "score"]].itertuples(index=False)
    }
    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_iter, users_who_answered_q, score_dict, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = users_who_answered_q[qs, :]  # numpy indexing
        answered_all = np.logical_and.reduce(user_sets_for_qs)
        answered_all = np.where(answered_all)[0]
        qs_total = [sum(score_dict[q, u] for q in qs) for u in answered_all]
        user_grand_total = [grand_totals[u] for u in answered_all]
        r = np.corrcoef(qs_total, user_grand_total)[0, 1]
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(
    k_corrset, compute_corrs, "NumPy bool_ array to identify users who answered qs"
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = np.zeros(
        (len(data.question.unique()), len(data.user.unique())), dtype=np.bool_
    )
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q, u] = True

    score_matrix = np.zeros(
        (len(data.user.unique()), len(data.question.unique())), dtype=np.float64
    )
    for q, u, s in data[["question", "user", "score"]].itertuples(index=False):
        score_matrix[u, q] = s

    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_matrix, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_iter, users_who_answered_q, score_matrix, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = users_who_answered_q[qs, :]
        answered_all = np.logical_and.reduce(user_sets_for_qs)
        answered_all = np.where(answered_all)[0]
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = [grand_totals[u] for u in answered_all]
        r = np.corrcoef(qs_total, user_grand_total)[0, 1]
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(
    k_corrset, compute_corrs, "Score matrix instead of dict to look up scores"
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    users_who_answered_q = np.zeros(
        (len(data.question.unique()), len(data.user.unique())), dtype=np.bool_
    )
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q, u] = True

    score_matrix = np.zeros(
        (len(data.user.unique()), len(data.question.unique())), dtype=np.float64
    )
    for q, u, s in data[["question", "user", "score"]].itertuples(index=False):
        score_matrix[u, q] = s

    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_matrix, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def corrcoef(a: list[float], b: list[float]) -> float | None:
    """same as np.corrcoef(a, b)[0, 1]"""
    n = len(a)
    sum_a = sum(a)
    sum_b = sum(b)
    sum_ab = sum(a_i * b_i for a_i, b_i in zip(a, b))
    sum_a_sq = sum(a_i**2 for a_i in a)
    sum_b_sq = sum(b_i**2 for b_i in b)
    num = n * sum_ab - sum_a * sum_b
    den = sqrt(n * sum_a_sq - sum_a**2) * sqrt(n * sum_b_sq - sum_b**2)
    if den == 0:
        return None
    return num / den


def compute_corrs(qs_iter, users_who_answered_q, score_matrix, grand_totals):
    result = []
    for qs in qs_iter:
        user_sets_for_qs = users_who_answered_q[qs, :]  # numpy indexing
        answered_all = np.logical_and.reduce(user_sets_for_qs)
        answered_all = np.where(answered_all)[0]
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = [grand_totals[u] for u in answered_all]
        r = corrcoef(qs_total, user_grand_total)
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(k_corrset, compute_corrs, "Custom corrcoef function")

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_iter = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    qs_iter = np.array(list(qs_iter))
    users_who_answered_q = np.zeros(
        (len(data.question.unique()), len(data.user.unique())), dtype=np.bool_
    )
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q, u] = True

    score_matrix = np.zeros(
        (len(data.user.unique()), len(data.question.unique())), dtype=np.float64
    )
    for q, u, s in data[["question", "user", "score"]].itertuples(index=False):
        score_matrix[u, q] = s

    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    corrs = compute_corrs(qs_iter, users_who_answered_q, score_matrix, grand_totals)
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame(corrs)
    return corrs, avg_iter_time_secs


def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    result = []
    for i in range(len(qs_combinations)):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs, :]  # numpy indexing
        answered_all = np.logical_and.reduce(user_sets_for_qs)
        answered_all = np.where(answered_all)[0]
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = [grand_totals[u] for u in answered_all]
        r = corrcoef(qs_total, user_grand_total)
        result.append({"qs": qs, "r": r})
    return result


run_benchmark(
    k_corrset, compute_corrs, "Pass qs_combinations as numpy array (prep for numba)"
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_combinations = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    qs_combinations = np.array(list(qs_combinations))
    users_who_answered_q = np.zeros(
        (len(data.question.unique()), len(data.user.unique())), dtype=np.bool_
    )
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q, u] = True

    score_matrix = np.zeros(
        (len(data.user.unique()), len(data.question.unique())), dtype=np.float64
    )
    for q, u, s in data[["question", "user", "score"]].itertuples(index=False):
        score_matrix[u, q] = s

    grand_totals = data.groupby("user").score.sum().rename("grand_total").to_dict()
    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


def corrcoef(a: list[float], b: list[float]) -> float | None:
    """same as np.corrcoef(a, b)[0, 1]"""
    n = len(a)
    sum_a = sum(a)
    sum_b = sum(b)
    sum_ab = sum(a_i * b_i for a_i, b_i in zip(a, b))
    sum_a_sq = sum(a_i**2 for a_i in a)
    sum_b_sq = sum(b_i**2 for b_i in b)
    num = n * sum_ab - sum_a * sum_b
    den = sqrt(n * sum_a_sq - sum_a**2) * sqrt(n * sum_b_sq - sum_b**2)
    if den == 0:
        return None
    return num / den


def compute_corrs(qs_iter, users_who_answered_q, score_matrix, grand_totals):
    result = np.empty(len(qs_iter), dtype=np.float64)
    for i in range(len(qs_iter)):
        qs = qs_iter[i]
        user_sets_for_qs = users_who_answered_q[qs, :]  # numpy indexing
        answered_all = np.logical_and.reduce(user_sets_for_qs)
        answered_all = np.where(answered_all)[0]
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = [grand_totals[u] for u in answered_all]
        result[i] = corrcoef(qs_total, user_grand_total)
    return result


run_benchmark(k_corrset, compute_corrs, "Result array instead of list (prep for numba)")

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data.user = data.user.map({u: i for i, u in enumerate(data.user.unique())})
    data.question = data.question.map(
        {q: i for i, q in enumerate(data.question.unique())}
    )
    qs_combinations = itertools.islice(
        itertools.combinations(data.question.unique(), K), max_iter
    )
    qs_combinations = np.array(list(qs_combinations))
    users_who_answered_q = np.zeros(
        (len(data.question.unique()), len(data.user.unique())), dtype=np.bool_
    )
    for q, u in data[["question", "user"]].itertuples(index=False):
        users_who_answered_q[q, u] = True

    score_matrix = np.zeros(
        (len(data.user.unique()), len(data.question.unique())), dtype=np.float64
    )
    for q, u, s in data[["question", "user", "score"]].itertuples(index=False):
        score_matrix[u, q] = s

    grand_totals = score_matrix.sum(axis=1)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit
def corrcoef_numba(a, b):
    """same as np.corrcoef(a, b)[0, 1]"""
    n = len(a)
    sum_a = sum(a)
    sum_b = sum(b)
    sum_ab = sum(a * b)
    sum_a_sq = sum(a * a)
    sum_b_sq = sum(b * b)
    num = n * sum_ab - sum_a * sum_b
    den = math.sqrt(n * sum_a_sq - sum_a**2) * math.sqrt(n * sum_b_sq - sum_b**2)
    return np.nan if den == 0 else num / den


@numba.njit(parallel=False)
def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    result = np.empty(len(qs_combinations), dtype=np.float64)
    for i in range(len(qs_combinations)):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs, :]
        # numba doesn't support np.logical_and.reduce
        answered_all = user_sets_for_qs[0]
        for j in range(1, len(user_sets_for_qs)):
            answered_all *= user_sets_for_qs[j]
        answered_all = np.where(answered_all)[0]
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef_numba(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "naive numba, with parallel=False, (no support for np.logical_and.reduce)",
    run_line_profiler=False,
)

In [ ]:
@numba.njit(parallel=True)
def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    result = np.empty(len(qs_combinations), dtype=np.float64)
    for i in numba.prange(len(qs_combinations)):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs, :]
        # numba doesn't support np.logical_and.reduce
        answered_all = user_sets_for_qs[0]
        for j in range(1, len(user_sets_for_qs)):
            answered_all *= user_sets_for_qs[j]
        answered_all = np.where(answered_all)[0]
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef_numba(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "naive numba, with parallel=True, (no support for np.logical_and.reduce)",
    run_line_profiler=False,
)

## Bring on the bitsets

In [ ]:
# TODO Let's go back and try bitsets, with our basic python code.
# or maybe not.. it's slow in normal python

In [ ]:
def bitset_to_list(arr):
    result = []
    for idx in range(arr.shape[0]):
        if arr[idx] == 0:
            continue
        for pos in range(64):
            if (arr[idx] & (np.int64(1) << np.int64(pos))) != 0:
                result.append(idx * 64 + pos)
    return np.array(result)


def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    result = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in range(num_qs):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs_combinations[i]]
        answered_all = np.bitwise_and.reduce(user_sets_for_qs)
        answered_all = bitset_to_list(answered_all)
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "bitsets, with no numba at all",
    run_line_profiler=True,
)

In [ ]:
@numba.njit
def bitset_to_list(arr):
    result = []
    for idx in range(arr.shape[0]):
        if arr[idx] == 0:
            continue
        for pos in range(64):
            if (arr[idx] & (np.int64(1) << np.int64(pos))) != 0:
                result.append(idx * 64 + pos)
    return np.array(result)


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    result = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in range(num_qs):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs_combinations[i]]
        answered_all = np.bitwise_and.reduce(user_sets_for_qs)
        answered_all = bitset_to_list(answered_all)
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "bitsets, with numba on bitset_to_list",
    run_line_profiler=True,
)

In [ ]:
@numba.njit
def bitset_to_list(arr):
    result = []
    for idx in range(arr.shape[0]):
        if arr[idx] == 0:
            continue
        for pos in range(64):
            if (arr[idx] & (np.int64(1) << np.int64(pos))) != 0:
                result.append(idx * 64 + pos)
    return np.array(result)


# @numba.njit
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


# @numba.njit
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    result = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in range(num_qs):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs_combinations[i]]
        answered_all = np.bitwise_and.reduce(user_sets_for_qs)
        answered_all = bitset_to_list(answered_all)
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef_numba(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba also on corrcoef",
    run_line_profiler=True,
)

In [ ]:
@numba.njit
def bitset_to_list(arr):
    result = []
    for idx in range(arr.shape[0]):
        if arr[idx] == 0:
            continue
        for pos in range(64):
            if (arr[idx] & (np.int64(1) << np.int64(pos))) != 0:
                result.append(idx * 64 + pos)
    return np.array(result)


# @numba.njit
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


# @numba.njit
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


@numba.njit
def bitset_and(arrays):
    result = arrays[0].copy()
    for i in range(1, len(arrays)):
        result &= arrays[i]
    return result


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    result = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in range(num_qs):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs_combinations[i]]
        answered_all = bitset_and(user_sets_for_qs)
        answered_all = bitset_to_list(answered_all)
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef_numba(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba also on bitset_and",
    run_line_profiler=True,
)

In [ ]:
@numba.njit
def bitset_to_list(arr):
    result = []
    for idx in range(arr.shape[0]):
        if arr[idx] == 0:
            continue
        for pos in range(64):
            if (arr[idx] & (np.int64(1) << np.int64(pos))) != 0:
                result.append(idx * 64 + pos)
    return np.array(result)


# @numba.njit
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


# @numba.njit
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


@numba.njit
def bitset_and(arrays):
    result = arrays[0].copy()
    for i in range(1, len(arrays)):
        result &= arrays[i]
    return result


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(parallel=False)
def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    result = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in range(num_qs):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs_combinations[i]]
        answered_all = bitset_and(user_sets_for_qs)
        answered_all = bitset_to_list(answered_all)
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef_numba(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "at this point numpy indexing slow, so numba on the func. parallel=False",
    run_line_profiler=False,
)

In [ ]:
@numba.njit
def bitset_to_list(arr):
    result = []
    for idx in range(arr.shape[0]):
        if arr[idx] == 0:
            continue
        for pos in range(64):
            if (arr[idx] & (np.int64(1) << np.int64(pos))) != 0:
                result.append(idx * 64 + pos)
    return np.array(result)


# @numba.njit
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


# @numba.njit
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


@numba.njit
def bitset_and(arrays):
    result = arrays[0].copy()
    for i in range(1, len(arrays)):
        result &= arrays[i]
    return result


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(parallel=True)
def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    result = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in numba.prange(num_qs):
        qs = qs_combinations[i]
        user_sets_for_qs = users_who_answered_q[qs_combinations[i]]
        answered_all = bitset_and(user_sets_for_qs)
        answered_all = bitset_to_list(answered_all)
        qs_total = score_matrix[answered_all, :][:, qs].sum(axis=1)
        user_grand_total = grand_totals[answered_all]
        result[i] = corrcoef_numba(qs_total, user_grand_total)
    return result


run_benchmark(
    k_corrset,
    compute_corrs,
    "at this point numpy indexing slow, so numba on the func. parallel=True",
    run_line_profiler=False,
)

In [ ]:
@numba.njit
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


@numba.njit
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(boundscheck=False, fastmath=True, parallel=False, nogil=True)
def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    corrs = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in numba.prange(num_qs):
        # bitset will contain users who answered all questions in qs_array[i]
        bitset = users_who_answered_q[qs_combinations[i, 0]].copy()
        for q in qs_combinations[i, 1:]:
            bitset &= users_who_answered_q[q]
        # retrieve stats for the users to compute correlation
        n = 0.0
        sum_a = 0.0
        sum_b = 0.0
        sum_ab = 0.0
        sum_a_sq = 0.0
        sum_b_sq = 0.0
        for idx in range(bitset_size):
            if bitset[idx] != 0:
                for pos in range(64):
                    if (bitset[idx] & (np.int64(1) << np.int64(pos))) != 0:
                        user_idx = idx * 64 + pos
                        score_for_qs = 0.0
                        for q in qs_combinations[i]:
                            score_for_qs += score_matrix[user_idx, q]
                        score_for_user = grand_totals[user_idx]
                        n += 1.0
                        sum_a += score_for_qs
                        sum_b += score_for_user
                        sum_ab += score_for_qs * score_for_user
                        sum_a_sq += score_for_qs * score_for_qs
                        sum_b_sq += score_for_user * score_for_user
        num = n * sum_ab - sum_a * sum_b
        den = np.sqrt(n * sum_a_sq - sum_a**2) * np.sqrt(n * sum_b_sq - sum_b**2)
        corrs[i] = np.nan if den == 0 else num / den
    return corrs


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba, bitsets, acculumation instead of arrays, inline, with parallel=False",
    run_line_profiler=False,
)

In [ ]:
@numba.njit(boundscheck=False, fastmath=True, nogil=True)
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


@numba.njit(boundscheck=False, fastmath=True, nogil=True)
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.int64
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    qs_combinations = []
    for i, qs in enumerate(itertools.combinations(all_qs, K)):
        if i == max_iter:
            break
        qs_combinations.append(qs)
    qs_combinations = np.array(qs_combinations)

    start = time.time()
    r_vals = compute_corrs(
        qs_combinations, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(qs) for qs in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(boundscheck=False, fastmath=True, parallel=True, nogil=True)
def compute_corrs(qs_combinations, users_who_answered_q, score_matrix, grand_totals):
    num_qs = qs_combinations.shape[0]
    bitset_size = users_who_answered_q[0].shape[0]
    corrs = np.empty(qs_combinations.shape[0], dtype=np.float64)
    for i in numba.prange(num_qs):
        # bitset will contain users who answered all questions in qs_array[i]
        bitset = users_who_answered_q[qs_combinations[i, 0]].copy()
        for q in qs_combinations[i, 1:]:
            bitset &= users_who_answered_q[q]
        # retrieve stats for the users and compute corrcoef
        n = 0.0
        sum_a = 0.0
        sum_b = 0.0
        sum_ab = 0.0
        sum_a_sq = 0.0
        sum_b_sq = 0.0
        for idx in range(bitset_size):
            if bitset[idx] != 0:
                for pos in range(64):
                    if (bitset[idx] & (np.int64(1) << np.int64(pos))) != 0:
                        score_for_qs = 0.0
                        for q in qs_combinations[i]:
                            score_for_qs += score_matrix[idx * 64 + pos, q]
                        score_for_user = grand_totals[idx * 64 + pos]
                        n += 1.0
                        sum_a += score_for_qs
                        sum_b += score_for_user
                        sum_ab += score_for_qs * score_for_user
                        sum_a_sq += score_for_qs * score_for_qs
                        sum_b_sq += score_for_user * score_for_user
        num = n * sum_ab - sum_a * sum_b
        den = np.sqrt(n * sum_a_sq - sum_a**2) * np.sqrt(n * sum_b_sq - sum_b**2)
        corrs[i] = np.nan if den == 0 else num / den
    return corrs


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba, bitsets, acculumation instead of arrays, inline, with parallel=True",
    run_line_profiler=False,
    num_iter=5_000_000,  # to match the original blog post
)

## Ok.. without precomputing Qs

In [ ]:
@numba.njit(boundscheck=False, fastmath=True, nogil=True)
def bitset_create(size):
    size_in_int64 = int(np.ceil(size / 64))
    return np.zeros(size_in_int64, dtype=np.int64)


@numba.njit(boundscheck=False, fastmath=True, nogil=True)
def bitset_add(arr, pos):
    int64_idx = pos // 64
    pos_in_int64 = pos % 64
    arr[int64_idx] |= np.int64(1) << np.int64(pos_in_int64)


def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.bool_
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    num_qs = all_qs.shape[0]
    start = time.time()
    r_vals, qs_combinations = compute_corrs(
        num_qs, max_iter, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(x) for x in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(boundscheck=False, fastmath=True, nogil=True)
def compute_combinations(num_qs, max_iter):
    qs_combinations = np.empty((max_iter, 5), dtype=np.int64)
    idx = 0
    for i in range(num_qs):
        for j in range(i + 1, num_qs):
            for k in range(j + 1, num_qs):
                for l in range(k + 1, num_qs):
                    for m in range(l + 1, num_qs):
                        qs_combinations[idx, 0] = i
                        qs_combinations[idx, 1] = j
                        qs_combinations[idx, 2] = k
                        qs_combinations[idx, 3] = l
                        qs_combinations[idx, 4] = m
                        idx += 1
                        if idx >= max_iter:
                            return qs_combinations


@numba.njit(boundscheck=False, fastmath=True, parallel=True, nogil=True)
def compute_corrs(num_qs, max_iter, users_who_answered_q, score_matrix, grand_totals):
    bitset_size = users_who_answered_q[0].shape[0]
    corrs = np.empty(max_iter, dtype=np.float64)
    qs_combinations = compute_combinations(num_qs, max_iter)
    for i in numba.prange(max_iter):
        # bitset will contain users who answered all questions in qs_array[i]
        bitset = users_who_answered_q[qs_combinations[i, 0]].copy()
        for q in qs_combinations[i, 1:]:
            bitset &= users_who_answered_q[q]
        # retrieve stats for the users and compute corrcoef
        n = 0.0
        sum_a = 0.0
        sum_b = 0.0
        sum_ab = 0.0
        sum_a_sq = 0.0
        sum_b_sq = 0.0
        for idx in range(bitset_size):
            if bitset[idx] != 0:
                for pos in range(64):
                    if (bitset[idx] & (np.int64(1) << np.int64(pos))) != 0:
                        score_for_qs = 0.0
                        for q in qs_combinations[i]:
                            score_for_qs += score_matrix[idx * 64 + pos, q]
                        score_for_user = grand_totals[idx * 64 + pos]
                        n += 1.0
                        sum_a += score_for_qs
                        sum_b += score_for_user
                        sum_ab += score_for_qs * score_for_user
                        sum_a_sq += score_for_qs * score_for_qs
                        sum_b_sq += score_for_user * score_for_user
        num = n * sum_ab - sum_a * sum_b
        den = np.sqrt(n * sum_a_sq - sum_a**2) * np.sqrt(n * sum_b_sq - sum_b**2)
        corrs[i] = np.nan if den == 0 else num / den
    return corrs, qs_combinations


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba, without precomputing qs",
    run_line_profiler=False,
    num_iter=5_000_000,  # to match the original blog post
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.bool_
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    num_qs = all_qs.shape[0]
    start = time.time()
    r_vals, qs_combinations = compute_corrs(
        num_qs, max_iter, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"qs": [tuple(x) for x in qs_combinations], "r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(boundscheck=False, fastmath=True, parallel=True, nogil=True)
def compute_corrs(num_qs, max_iter, users_who_answered_q, score_matrix, grand_totals):
    bitset_size = users_who_answered_q[0].shape[0]
    corrs = np.empty(max_iter, dtype=np.float64)

    # Compute combinations inline
    qs_combinations = np.empty((max_iter, 5), dtype=np.int64)
    try:
        idx = 0
        for i in range(num_qs):
            for j in range(i + 1, num_qs):
                for k in range(j + 1, num_qs):
                    for l in range(k + 1, num_qs):
                        for m in range(l + 1, num_qs):
                            qs_combinations[idx, 0] = i
                            qs_combinations[idx, 1] = j
                            qs_combinations[idx, 2] = k
                            qs_combinations[idx, 3] = l
                            qs_combinations[idx, 4] = m
                            idx += 1
                            if idx >= max_iter:
                                raise Exception
    except Exception:
        pass

    for i in numba.prange(max_iter):
        # bitset will contain users who answered all questions in qs_array[i]
        bitset = users_who_answered_q[qs_combinations[i, 0]].copy()
        for q in qs_combinations[i, 1:]:
            bitset &= users_who_answered_q[q]
        # retrieve stats for the users and compute corrcoef
        n = 0.0
        sum_a = 0.0
        sum_b = 0.0
        sum_ab = 0.0
        sum_a_sq = 0.0
        sum_b_sq = 0.0
        for idx in range(bitset_size):
            if bitset[idx] != 0:
                for pos in range(64):
                    if (bitset[idx] & (np.int64(1) << np.int64(pos))) != 0:
                        score_for_qs = 0.0
                        for q in qs_combinations[i]:
                            score_for_qs += score_matrix[idx * 64 + pos, q]
                        score_for_user = grand_totals[idx * 64 + pos]
                        n += 1.0
                        sum_a += score_for_qs
                        sum_b += score_for_user
                        sum_ab += score_for_qs * score_for_user
                        sum_a_sq += score_for_qs * score_for_qs
                        sum_b_sq += score_for_user * score_for_user
        num = n * sum_ab - sum_a * sum_b
        den = np.sqrt(n * sum_a_sq - sum_a**2) * np.sqrt(n * sum_b_sq - sum_b**2)
        corrs[i] = np.nan if den == 0 else num / den
    return corrs, qs_combinations


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba, computing qs inline",
    run_line_profiler=False,
    num_iter=5_000_000,  # to match the original blog post
)

In [ ]:
def k_corrset(data, K, max_iter=1000):
    data = data.copy()
    data["user"] = data["user"].map({u: i for i, u in enumerate(data.user.unique())})
    data["question"] = data["question"].map(
        {q: i for i, q in enumerate(data.question.unique())}
    )

    all_qs = data.question.unique()
    grand_totals = data.groupby("user").score.sum().values

    # create bitsets
    users_who_answered_q = np.array(
        [bitset_create(data.user.nunique()) for _ in range(data.question.nunique())]
    )
    for q, u in data[["question", "user"]].values:
        bitset_add(users_who_answered_q[q], u)

    score_matrix = np.zeros(
        (data.user.nunique(), data.question.nunique()), dtype=np.bool_
    )
    for row in data.itertuples():
        score_matrix[row.user, row.question] = row.score

    # todo, would be nice to have a super fast iterator / generator in numba
    # rather than creating the whole array
    num_qs = all_qs.shape[0]
    start = time.time()
    r_vals = compute_corrs(
        num_qs, max_iter, users_who_answered_q, score_matrix, grand_totals
    )
    avg_iter_time_secs = (time.time() - start) / max_iter
    corrs = pd.DataFrame({"r": r_vals})
    return corrs, avg_iter_time_secs


@numba.njit(boundscheck=False, fastmath=True, parallel=False, nogil=True)
def compute_corrs(num_qs, max_iter, users_who_answered_q, score_matrix, grand_totals):
    bitset_size = users_who_answered_q[0].shape[0]
    corrs = np.empty(max_iter, dtype=np.float64)
    # Compute combinations inline
    outer_idx = 0
    for i in range(num_qs):
        for j in range(i + 1, num_qs):
            for k in range(j + 1, num_qs):
                for l in numba.prange(k + 1, num_qs):
                    for m in numba.prange(l + 1, num_qs):
                        qs_combination = np.array([i, j, k, l, m])
                        # bitset will contain users who answered all questions in qs_array[i]
                        bitset = users_who_answered_q[qs_combination[0]].copy()
                        for q in qs_combination:
                            bitset &= users_who_answered_q[q]
                        # retrieve stats for the users and compute corrcoef
                        n = 0.0
                        sum_a = 0.0
                        sum_b = 0.0
                        sum_ab = 0.0
                        sum_a_sq = 0.0
                        sum_b_sq = 0.0
                        for idx in range(bitset_size):
                            if bitset[idx] != 0:
                                for pos in range(64):
                                    if (
                                        bitset[idx] & (np.int64(1) << np.int64(pos))
                                    ) != 0:
                                        score_for_qs = 0.0
                                        for q in qs_combination:
                                            score_for_qs += score_matrix[
                                                idx * 64 + pos, q
                                            ]
                                        score_for_user = grand_totals[idx * 64 + pos]
                                        n += 1.0
                                        sum_a += score_for_qs
                                        sum_b += score_for_user
                                        sum_ab += score_for_qs * score_for_user
                                        sum_a_sq += score_for_qs * score_for_qs
                                        sum_b_sq += score_for_user * score_for_user
                        num = n * sum_ab - sum_a * sum_b
                        den = np.sqrt(n * sum_a_sq - sum_a**2) * np.sqrt(
                            n * sum_b_sq - sum_b**2
                        )
                        corrs[outer_idx] = np.nan if den == 0 else num / den
                        outer_idx += 1
                        if outer_idx >= max_iter:
                            return corrs


run_benchmark(
    k_corrset,
    compute_corrs,
    "numba, combinations inline",
    run_line_profiler=False,
    # num_iter=10,  # to match the original blog post
)